In [1]:
import pandas as pd
import os
import scipy.stats as stats
import scikit_posthocs as sp



In [2]:
path = '/export/usuarios_ml4ds/ammesa/Data/4_indexed_data/stats_folder'
df_weighted = pd.read_excel(os.path.join(path, 'metrics_retrieval_CI.xlsx'))
df_re = pd.read_csv(os.path.join(path, 'metrics_retrieval_def.csv'), delimiter=";")

In [3]:
def get_significance_marker( p_value):
    if p_value < 0.001:
        return "***"
    elif p_value < 0.01:
        return "**"
    elif p_value < 0.05:
        return "*"
    else:
        return ""
    
def format_metric(metric):
    return (
            "MRR@3" if metric == "mrr_3" else
            "MRR@5" if metric == "mrr_5" else
            "NDCG@3" if metric == "ndcg_3" else
            "NDCG@5" if metric == "ndcg_5" else
            "Precision@3" if metric == "precision_3" else
            "Precision@5" if metric == "precision_5" else
            "Recall@3" if metric == "recall_3" else
            "Recall@5" if metric == "recall_5" else
            "RHit@3" if metric == "rank_hit_3" else
            "RHit@5" if metric == "rank_hit_5" else
            "Time (s)" if "time" in metric else
            metric  # Fallback si no está en la lista
        )
        

def check_normality( values):
    stat, p_value = stats.shapiro(values)
    return p_value > 0.05  

def perform_anova( groups):
    f_stat, p_value = stats.f_oneway(*groups)
    return f_stat, p_value

def perform_kruskal_wallis( groups):
    h_stat, p_value = stats.kruskal(*groups)
    return h_stat, p_value

# Normality tests:

In [4]:
df_re = pd.read_excel(os.path.join(path, 'metrics_retrieval_def.xlsx'))
df_re

,retrieval_method,avg_precision@3_,avg_recall@3_,avg_ndcg@3_,avg_hit@3_,avg_rank_hit@3_,avg_precision@5_,avg_recall@5_,avg_ndcg@5_,avg_hit@5_,avg_rank_hit@5_,avg_precision@10_,avg_recall@10_,avg_ndcg@10_,avg_hit@10_,avg_rank_hit@10_,thr,weighted
0,TB_ANN,0.0395,0.1185,0.0853,0.1185,0.0740,0.0387,0.1936,0.1160,0.1936,0.0909,0.0384,0.3837,0.1767,0.3837,0.1154,0,True
1,TB_ANN,0.0395,0.1186,0.0854,0.1186,0.0741,0.0388,0.1937,0.1161,0.1937,0.0910,0.0384,0.3837,0.1767,0.3837,0.1154,_dynamic,True
2,TB_ANN,0.0428,0.1284,0.0923,0.1284,0.0800,0.0419,0.2094,0.1254,0.2094,0.0982,0.0415,0.4146,0.1908,0.4146,0.1246,0,False
3,TB_ANN,0.0430,0.1290,0.0927,0.1290,0.0803,0.0421,0.2103,0.1259,0.2103,0.0986,0.0417,0.4167,0.1917,0.4167,0.1252,_dynamic,False
4,TB_ENN,0.0793,0.2378,0.1706,0.2378,0.1476,0.0782,0.3908,0.2332,0.3908,0.1820,0.0778,0.7780,0.3566,0.7780,0.2319,0,False
5,TB_ENN,0.0797,0.2391,0.1716,0.2391,0.1485,0.0785,0.3927,0.2344,0.3927,0.1830,0.0783,0.7829,0.3587,0.7829,0.2333,_dynamic,False
6,TB_ENN,0.0803,0.2410,0.1732,0.2410,0.1499,0.0793,0.3967,0.2368,0.3967,0.1850,0.0790,0.7902,0.3622,0.7902,0.2356,_dynamic,True
7,TB_ENN,0.0804,0.2411,0.1732,0.2411,0.1500,0.0793,0.3967,0.2368,0.3967,0.1850,0.0790,0.7903,0.3623,0.7903,0.2357,0,True
8,ENN,0.0790,0.2371,0.1699,0.2371,0.1469,0.0778,0.3890,0.2320,0.3890,0.1811,0.0774,0.7739,0.3547,0.7739,0.2307,_dynamic,False
9,ANN,0.0455,0.1365,0.0980,0.1365,0.0849,0.0451,0.2253,0.1343,0.2253,0.1048,0.0448,0.4482,0.2054,0.4482,0.1335,0,True


In [5]:
df_weighted

,FileID,Method,Threshold,Weight,Metric,Weighted Mean,95% CI
0,10_04_25_questions,TB_ANN,0,True,mrr@3,0.073988,0.000958
1,10_04_25_questions,TB_ANN,0,True,mrr@5,0.090890,0.000968
2,10_04_25_questions,TB_ANN,0,True,mrr@10,0.115393,0.000951
3,10_04_25_questions,TB_ANN,0,True,precision@3,0.039498,0.000460
4,10_04_25_questions,TB_ANN,0,True,precision@5,0.038718,0.000337
...,...,...,...,...,...,...,...
175,10_04_25_questions,ANN,none,none,hit@5,0.225273,0.001782
176,10_04_25_questions,ANN,none,none,hit@10,0.448226,0.002121
177,10_04_25_questions,ANN,none,none,rank_hit@3,0.084850,0.001016
178,10_04_25_questions,ANN,none,none,rank_hit@5,0.104823,0.001023


In [6]:
df_weighted['Method'] = df_weighted['Method'].astype(str) + '_' + df_weighted['Threshold'].astype(str) + '_' + df_weighted['Weight'].astype(str)
df_weighted = df_weighted.drop(columns=['Threshold','Weight'])

In [7]:
df_weighted

,FileID,Method,Metric,Weighted Mean,95% CI
0,10_04_25_questions,TB_ANN_0_True,mrr@3,0.073988,0.000958
1,10_04_25_questions,TB_ANN_0_True,mrr@5,0.090890,0.000968
2,10_04_25_questions,TB_ANN_0_True,mrr@10,0.115393,0.000951
3,10_04_25_questions,TB_ANN_0_True,precision@3,0.039498,0.000460
4,10_04_25_questions,TB_ANN_0_True,precision@5,0.038718,0.000337
...,...,...,...,...,...
175,10_04_25_questions,ANN_none_none,hit@5,0.225273,0.001782
176,10_04_25_questions,ANN_none_none,hit@10,0.448226,0.002121
177,10_04_25_questions,ANN_none_none,rank_hit@3,0.084850,0.001016
178,10_04_25_questions,ANN_none_none,rank_hit@5,0.104823,0.001023


In [8]:
df_re['Method'] = df_re['retrieval_method'].astype(str) + '_' + df_re['weighted'].astype(str)# + '_' + df_re['thr'].astype(str) + 

# Step 2: Drop the old method and modifiers if you don't need them
df_re = df_re.drop(columns=['retrieval_method', 'thr', 'weighted'])

# Step 3: Melt the dataframe so that metric names become a column, and values are aligned accordingly
df_melted = df_re.melt(id_vars=['Method'], var_name='Metric', value_name='Value')

df_melted

,Method,Metric,Value
0,TB_ANN_True,avg_precision@3_,0.0395
1,TB_ANN_True,avg_precision@3_,0.0395
2,TB_ANN_False,avg_precision@3_,0.0428
3,TB_ANN_False,avg_precision@3_,0.0430
4,TB_ENN_False,avg_precision@3_,0.0793
...,...,...,...
145,TB_ENN_False,avg_rank_hit@10_,0.2333
146,TB_ENN_True,avg_rank_hit@10_,0.2356
147,TB_ENN_True,avg_rank_hit@10_,0.2357
148,ENN_False,avg_rank_hit@10_,0.2307


In [14]:
def format_weighted_value(row):
    mean = f"{row['Weighted Mean']:.3f}"
    ci = f"{row['95% CI']:.3f}"
    value = f"{mean} \pm {ci}"

    metric = row["Metric"]
    method = row["Method"]

    """
    # Agregar '*' si Kruskal-Wallis indica diferencias significativas
    if metric in significance_dict["Kruskal-Wallis p-value"]:
        kw_p_value = significance_dict["Kruskal-Wallis p-value"][metric]
        if kw_p_value < 0.05:
            value += " *"  # Indica que hay diferencias globales
    """
    significance_marker = ""
    if metric in significance_dict["Significant post-hoc tests (Dunn)"]:
        dunn_df = significance_dict["Significant post-hoc tests (Dunn)"][metric]
        import pdb; pdb.set_trace()
        for other_method in dunn_df.index:
            if method in dunn_df.columns and other_method in dunn_df.index:
                p_val = dunn_df.loc[other_method, method]
                if p_val < 0.001:
                    significance_marker = "***"
                elif p_val < 0.01:
                    significance_marker = "**"
                elif p_val < 0.05:
                    significance_marker = "*"

    value += f"^{{{significance_marker}}}"

    return f"\\({value}\\)"

df_long = df_melted
df_weighted['Weighted Mean'] = pd.to_numeric(df_weighted['Weighted Mean'], errors='coerce')
df_weighted['95% CI'] = pd.to_numeric(df_weighted['95% CI'], errors='coerce')
normality_tests = {}
for metric in df_long["Metric"].unique():
    for method in df_long["Method"].unique():
        values = df_long[(df_long["Metric"] == metric) & (df_long["Method"] == method)]["Value"]
        if len(values) > 3:  # Shapiro necesita al menos unos pocos valores
            stat, p_value = stats.shapiro(values)
            normality_tests[(metric, method)] = p_value

# ANOVA or Kruskal-Wallis
anova_results = {}
kruskal_results = {}

for metric in df_long["Metric"].unique():
    groups = [df_long[(df_long["Metric"] == metric) & (df_long["Method"] == method)]["Value"].dropna() for method in df_long["Method"].unique()]
    # if normality in at least one group
    normal = all(p > 0.05 for p in [normality_tests.get((metric, method), 0) for method in df_long["Method"].unique()])
    
    if normal:
        stat, p_value = stats.f_oneway(*groups)
        anova_results[metric] = p_value
    else:
        stat, p_value = stats.kruskal(*groups)
        kruskal_results[metric] = p_value

# post-hoc tests if Kruskal-Wallis is significant
posthoc_results = {}
for metric, p in kruskal_results.items():
    if p < 0.05:  # Si hay diferencias significativas
        df_filtered = df_long[df_long["Metric"] == metric]
        posthoc = sp.posthoc_dunn(df_filtered, val_col="Value", group_col="Method", p_adjust="bonferroni")
        posthoc_results[metric] = posthoc


    significance_dict = {
        "ANOVA p-value": anova_results,
        "Kruskal-Wallis p-value": kruskal_results,
        "Significant post-hoc tests (Dunn)": posthoc_results
    }
    
    df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
    
    df_weighted_pivot = df_weighted.pivot_table(index=["FileID", "Method"], columns="Metric", values="Weighted Mean ± CI", aggfunc="first")
    
    latex_output = "\\begin{table*}[h]\n\\centering\n\\resizebox{\\textwidth}{!}{%\n\\begin{tabular}{lc" + "c" * len(df_weighted_pivot.columns) + "}\n\\arrayrulecolor{black}\n\\toprule\n"
    
    latex_output += "File & \\textbf{Method} & " + " & ".join([f"\\textbf{{{col}}}" for col in df_weighted_pivot.columns]) + " \\\\\n\\midrule\n"
    
    current_file_id = None 
    for (file_idx, method), row in df_weighted_pivot.iterrows():
        if file_idx != current_file_id:  
            if current_file_id is not None:
                latex_output += "\\arrayrulecolor{black}\\specialrule{0.5pt}{0pt}{0pt}\\arrayrulecolor{black}\n"

            col_n = str(len(df_weighted_pivot.columns) + 2)
            file_id = str(file_idx+'1')
            
            latex_output += f"\\multicolumn{{{col_n}}}{{l}}{{\\textbf{{File {file_id}}}}} \\\\\n"
            
            latex_output += "\\arrayrulecolor{black}\\specialrule{0.5pt}{0pt}{0pt}\\arrayrulecolor{black}\n"
            
            current_file_id = file_idx  
        
        row_start = "\\rowcolor{tableblue} " if method == "XX" else ""
        
        latex_output += row_start + f"& \\textbf{{{method}}} & " + " & ".join(row.astype(str)) + " \\\\\n"
    
    latex_output += "\\bottomrule\n\\end{tabular}}\n\\caption{Performance Metrics per Method for Each File (Best values in bold)}\n\\label{tab:grouped_results}\n\\end{table*}"
    
    print(latex_output)


\begin{table*}[h]
\centering
\resizebox{\textwidth}{!}{%
\begin{tabular}{lccccccccccccccccccc}
\arrayrulecolor{black}
\toprule
File & \textbf{Method} & \textbf{hit@10} & \textbf{hit@3} & \textbf{hit@5} & \textbf{mrr@10} & \textbf{mrr@3} & \textbf{mrr@5} & \textbf{ndcg@10} & \textbf{ndcg@3} & \textbf{ndcg@5} & \textbf{precision@10} & \textbf{precision@3} & \textbf{precision@5} & \textbf{rank_hit@10} & \textbf{rank_hit@3} & \textbf{rank_hit@5} & \textbf{recall@10} & \textbf{recall@3} & \textbf{recall@5} \\
\midrule
\multicolumn{20}{l}{\textbf{File 10_04_25_questions1}} \\
\arrayrulecolor{black}\specialrule{0.5pt}{0pt}{0pt}\arrayrulecolor{black}
& \textbf{ANN_none_none} & \(0.448 \pm 0.002^{}\) & \(0.137 \pm 0.001^{}\) & \(0.225 \pm 0.002^{}\) & \(0.134 \pm 0.001^{}\) & \(0.085 \pm 0.001^{}\) & \(0.105 \pm 0.001^{}\) & \(0.205 \pm 0.001^{}\) & \(0.098 \pm 0.001^{}\) & \(0.134 \pm 0.001^{}\) & \(0.045 \pm 0.000^{}\) & \(0.046 \pm 0.000^{}\) & \(0.045 \pm 0.000^{}\) & \(0.134 \pm 0.001^{}\)

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipykernel_965116/1275459928.py:4: SyntaxWarning: invalid escape sequence '\p'
  value = f"{mean} \pm {ci}"


In [17]:
aux_q = pd.read_excel('/export/usuarios_ml4ds/ammesa/Data/question_bank/10_04_25_questions/pub_med_revised.xlsx')
aux_q

,Unnamed: 0.1,Unnamed: 0,id,id_preproc,full_doc,passage,question,relevant_docs,subqueries
0,0,5,25440440,25440440,Hidradenitis suppurativa (HS) is a chronic inf...,Hidradenitis suppurativa (HS) is a chronic inf...,Is hidradenitis suppurativa a systemic disease...,25440440,"['""hidradenitis suppurativa prevalence'\n ' hi..."
1,1,4,25432519,25432519,Tumor microenvironment immunity is associated ...,Tumor microenvironment immunity is associated ...,Do tumor-infiltrating immune cell profiles and...,25432519,"['""tumor microenvironment immunity and breast ..."
2,2,3,25431941,25431941,This study examined links between DNA methylat...,This study examined links between DNA methylat...,Is methylation of the FGFR2 gene associated wi...,25431941,"['""FGFR2 gene methylation and birth weight ass..."
3,3,0,25429730,25429730,Chronic rhinosinusitis (CRS) is a heterogeneou...,Chronic rhinosinusitis (CRS) is a heterogeneou...,Are group 2 innate lymphoid cells ( ILC2s ) in...,25429730,"['""Chronic rhinosinusitis ILC2 levels' ' ILC2 ..."
4,4,2,25445714,25445714,Psammaplin A (PsA) is a natural product isolat...,Psammaplin A (PsA) is a natural product isolat...,Does psammaplin A induce Sirtuin 1-dependent a...,25445714,"['""Psammaplin A' ' Sirtuin 1-dependent autopha..."
...,...,...,...,...,...,...,...,...,...
211260,211264,211263,8219523,8219523,To evaluate the effects of different frequenci...,To evaluate the effects of different frequenci...,Does sine-wave auricular TENS produce frequenc...,8219523,"['""sine-wave auricular TENS frequency effect o..."
211261,211265,211266,8205673,8205673,There is increasing recognition of myocardial ...,There is increasing recognition of myocardial ...,Does intracoronary angiotensin-converting enzy...,8205673,"['""ACE inhibition' ' diastolic function improv..."
211262,211266,211267,8215873,8215873,To determine if splenectomy results in an incr...,To determine if splenectomy results in an incr...,Does transfusion significantly increase the ri...,8215873,"['""transfusion risk for infection after spleni..."
211263,211267,211265,8204319,8204319,This study set out to show how well samples fr...,This study set out to show how well samples fr...,Does evaluation of the use of general practice...,8204319,"['""general practice age-sex registers accuracy..."
